In [4]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import RegexpTokenizer
import networkx as nx
from nltk.stem.snowball import RussianStemmer
from itertools import combinations
import pickle


импорт **необходимых бибилотек**


In [1]:
def similarity(s1, s2): #
    if not len(s1) or not len(s2):
        return 0.0
    return len(s1.intersection(s2)) / (1.0 * (len(s1) + len(s2)))


вычисляет **сходство** предложений



In [2]:
def textrank(text):
    sentences = sent_tokenize(text)
    tokenizer = RegexpTokenizer(r'\w+')
    lmtzr = RussianStemmer()
    words = [set(lmtzr.stem(word) for word in tokenizer.tokenize(sentence.lower()))
             for sentence in sentences]

    pairs = combinations(range(len(sentences)), 2)
    scores = [(i, j, similarity(words[i], words[j])) for i, j in pairs]
    scores = filter(lambda x: x[2], scores)

    g = nx.Graph()
    g.add_weighted_edges_from(scores)
    pr = nx.pagerank(g)

    return sorted(((i, pr[i], s) for i, s in enumerate(sentences) if i in pr),
                  key=lambda x: pr[x[0]], reverse=True)


оценивает **важность каждого предложения** в наборе


In [3]:
def extract(text, n=5):
    tr = textrank(text)
    top_n = sorted(tr[:n])
    return ' '.join(x[2] for x in top_n)


топ **n самых важных предложений** в наборе


In [9]:
data = pickle.load(open("bankir_ru_news.pickle","rb"))
text = data[13]['text']
text

'В Астраханской области неизвестные взорвали банкомат Сбербанка. Для этого преступники использовали баллон с газом. Однако, как сообщает информагентство «Астрахань 24», кассету с денежными средствами подозреваемые извлечь не смогли, так как ее повредило ударной волной. По данным регионального УМВД, в настоящее время на месте происшествия работают оперативные службы. Лица, совершившие преступления, пока не выявлены. Банк подсчитывает причиненный ущерб. Представители банка сообщили, что инцидент подтверждается. Пострадавших нет. В настоящее время по данному факту ведутся следственные действия. В интересах следствия подробности не раскрываются. фото с сайта «Астрахань 24»'

In [13]:
desc = extract(text,3)
desc

'По данным регионального УМВД, в настоящее время на месте происшествия работают оперативные службы. В настоящее время по данному факту ведутся следственные действия. В интересах следствия подробности не раскрываются.'